<a href="https://colab.research.google.com/github/suyashchaturvedi26/Warehouse-location-Problem/blob/main/WLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()  # for plot styling
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from pandas import DataFrame
from pulp import *
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
R1_df=pd.read_excel('OR_Data.xlsx',sheet_name ='R1').set_index('N')
R2_df=pd.read_excel('OR_Data.xlsx',sheet_name ='R2').set_index('N')
R3_df=pd.read_excel('OR_Data.xlsx',sheet_name ='R3').set_index('N')
R4_df=pd.read_excel('OR_Data.xlsx',sheet_name ='R4').set_index('N')
R5_df=pd.read_excel('OR_Data.xlsx',sheet_name ='R5').set_index('N')
R6_df=pd.read_excel('OR_Data.xlsx',sheet_name ='R6').set_index('N')

Warehouse=['W1','W2','W3']

In [ ]:
plt.style.use('default')

x=np.array(R6_df['N_X'])
y=np.array(R6_df['N_Y'])

plt.figure(figsize=(15, 10))
plt.scatter(x, y, s=200, cmap='viridis',c='orange')

plt.title("Customer's Coordinates",fontsize=20, fontweight='bold')
plt.xlabel('X Co-ordinates',fontsize=20)
plt.ylabel('Y Co-ordinates',fontsize=20)

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=99)

X = np.array(R6_df.drop(['a'], 1).astype(float))

In [ ]:
wt_kmeansclus = kmeans.fit(X)
predicted_kmeans = kmeans.predict(X)

In [ ]:
plt.style.use('default')
plt.figure(figsize=(15, 10))

plt.scatter(X[:,0], X[:,1], c=wt_kmeansclus.labels_.astype(float),s=100,cmap='tab20b',marker="o")

centers = wt_kmeansclus.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='Black', s=500, marker= '^',alpha=0.8);

In [ ]:
distance_matrix=euclidean_distances(X, centers)

demands=R6_df['a']

weighted_demands_df=pd.DataFrame((distance_matrix * demands[:,None]),columns=Warehouse,index=R6_df.index)

weighted_demands=weighted_demands_df.to_dict(orient='index')

In [ ]:
res = dict(zip(Warehouse, weighted_demands)) 

In [ ]:
# Initializing Model
model=LpProblem('Warehouse Location Problem', LpMinimize)

In [ ]:
Routes = [(cust,war) for cust in R6_df.index for war in Warehouse]

In [ ]:
bool_vars=LpVariable.dicts('Customer',(R6_df.index,Warehouse),0,None,LpBinary)

In [ ]:
model += lpSum([bool_vars[cust][war]*weighted_demands[cust][war] for (cust,war) in Routes]), "Sum of Weighted"

In [ ]:
for cust in R6_df.index:
    model += lpSum([bool_vars[cust][war] for war in Warehouse]) == 1, "Sum of Products out of Warehouse %s"%cust

In [ ]:
model.solve()

In [ ]:
print("Status:", LpStatus[model.status])

In [ ]:
model.writeLP("Warehouse Location Problem.lp")

In [ ]:
for v in model.variables():
    if(v.varValue==1):
        print(v.name, "=", v.varValue)

In [ ]:
print("Sum of Weighted Distances of the Customers = ", value(model.objective))